# Radiosonde Data Plots

includes large dataset plots with holoviews...

version control: 
* v1.0 initial release
* v1.1 added comments and matplotlib figure size

In [ ]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [10, 5]
import igra #https://pypi.org/project/igra/

In [ ]:
# constants and conversion factors

m2ft = 3.28084

In [ ]:
# download Edwards AFB to local drive
igra.download.station("USM00072381", "./")

In [ ]:
# read back the data from local drive into pandas dataframe

data, station = igra.read.ascii_to_dataframe("./USM00072381-data.txt.zip")

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
# quick pressure plot

plt.scatter(data['pres'], data['gph'])
plt.show()

In [ ]:
# a little better pressure plot

plt.scatter(data['pres'], data['gph'], s=0.05, marker='.', 
            c="#36b5d8")
plt.grid(visible=True, alpha=0.2)
plt.show()

In [ ]:
# temperature plot

plt.scatter(data['temp'], data['gph'], s=0.05, marker='.', 
            c="#36b5d8")
plt.grid(visible=True, alpha=0.2)
plt.xlabel('temp [C]')
plt.ylabel('gph [m]')
plt.show()

In [ ]:
#pip install holoviews, hvplot, datashader

In [ ]:
# import the pro stuff

import hvplot.pandas
import holoviews as hv, colorcet
hv.extension('bokeh')
from bokeh.themes import built_in_themes
hv.renderer('bokeh').theme = built_in_themes['dark_minimal']

In [ ]:
# temperature plot 

data.hvplot.scatter(x='temp', y='gph', rasterize=True, cnorm='log', colorbar=False, 
                  xlabel='temp [C]', ylabel='gph [m]', grid=True, cmap=colorcet.kbc,
                  ylim=(0,40000))

Note data structures and clear patter to collect temperature at set heights.

In [ ]:
# we already have Edwards, but let's keep it complete
# the order relates to latitude
dataset = {}

stations = {'Fairbanks':'USM00070261',
            'Edwards':'USM00072381',
            'Florianopolis':'BRM00083899'}

for station, sta_ID in stations.items():
    #igra.download.station(sta_ID, "./")
    data, _ = igra.read.ascii_to_dataframe(f'{sta_ID}-data.txt.zip')
    dataset[station] = data

In [ ]:
# create composite plot

sta_y = '1940-05-01' # start of date interval
end_y = '2022-09-01' # end of date interval

fig, axs = plt.subplots(2, 3, sharex='row',sharey='row')
col_idx = 0

for station, data in dataset.items():
    # first slice the data to include only data between the start and end dates
    filt_data = data[(data.index>sta_y) & (data.index<end_y)].loc[:,['temp', 'pres', 'gph']].dropna()
    
    # pressure plots
    d_label = 'pres'
    x_data = filt_data[d_label] / 1000 # [kPa]
    y_data = filt_data['gph'] * m2ft / 1000 # [kft]
    axs[0, col_idx].scatter(x_data, y_data, s=0.05, marker='.', 
            c="#36b5d8", alpha=0.9)
    axs[0, col_idx].grid(visible=True, alpha=0.2)
    axs[0, col_idx].set_title(station)
    if col_idx == 0:
        axs[0, col_idx].set_xlabel(f'{d_label} [kPa]')
        axs[0, col_idx].xaxis.set_label_coords(-0.12, -0.06)
        axs[0, col_idx].set_ylabel('gph [kft]')
        
    # temp plots
    d_label = 'temp'
    x_data = filt_data[d_label] # [C]
    y_data = filt_data['gph'] * m2ft / 1000 # [kft]
    axs[1, col_idx].scatter(x_data, y_data, s=0.05, marker='.', 
            c="#36b5d8", alpha=0.9)
    axs[1, col_idx].grid(visible=True, alpha=0.2)
    #axs[1, col_idx].set_title(station)
    if col_idx == 0:
        axs[1, col_idx].set_xlabel(f'{d_label} [C]')
        axs[1, col_idx].xaxis.set_label_coords(-0.12, -0.06)
        axs[1, col_idx].set_ylabel('gph [kft]')
    col_idx += 1

plt.show()